**This notebook is an exercise in the [Advanced SQL](https://www.kaggle.com/learn/advanced-sql) course.  You can reference the tutorial at [this link](https://www.kaggle.com/alexisbcook/analytic-functions).**

---


# Introduction

Here, you'll use window functions to answer questions about the [Chicago Taxi Trips](https://www.kaggle.com/chicago/chicago-taxi-trips-bq) dataset.

Before you get started, run the code cell below to set everything up.

In [12]:
# Get most recent checking code
!pip install -U -t /kaggle/working/ git+https://github.com/Kaggle/learntools.git
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql_advanced.ex2 import *
print("Setup Complete")

  Cloning https://github.com/Kaggle/learntools.git to /tmp/pip-req-build-34ycvd93
  Running command git clone --filter=blob:none --quiet https://github.com/Kaggle/learntools.git /tmp/pip-req-build-34ycvd93
  Resolved https://github.com/Kaggle/learntools.git to commit 7a83948d1820d0282e518fc44692bb3118c9e1a6
  Preparing metadata (setup.py) ... done
  Created wheel for learntools: filename=learntools-0.3.4-py3-none-any.whl size=268963 sha256=916f9eb3ebfdfedfbaf16c71d267da20f46f8b1b429f0c7469b012aa1f785f4e
  Stored in directory: /tmp/pip-ephem-wheel-cache-n62l8s6w/wheels/2f/6c/3c/aa9f50cfb5a862157cb4c7a5b34881828cf45404698255dced
Successfully built learntools
Setup Complete


The following code cell fetches the `taxi_trips` table from the `chicago_taxi_trips` dataset. We also preview the first five rows of the table.  You'll use the table to answer the questions below.

In [13]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "chicago_taxi_trips" dataset
dataset_ref = client.dataset("chicago_taxi_trips", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "taxi_trips" table
table_ref = dataset_ref.table("taxi_trips")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,883990e9327a3bca3428d5f519a95d03a57945a0,dff52b318380d1870e4fa113fbdf19ba3c85e23a9b2d26...,2013-08-22 11:30:00+00:00,2013-08-22 11:30:00+00:00,0,0.0,1.703184e+10,1.703184e+10,28,28,...,0.0,3.65,Cash,Choice Taxi Association,41.870415,-87.675086,POINT (-87.6750856208 41.8704150003),41.870415,-87.675086,POINT (-87.6750856208 41.8704150003)
1,cae7ec1ba2ef76f4a922ca57be7724a936cc2f80,64848e03709152785ab1f99953fe601c44acad3e49fc5a...,2013-08-19 02:15:00+00:00,2013-08-19 02:30:00+00:00,1200,10.6,NaN,NaN,1,24,...,1.0,24.25,Cash,Dispatch Taxi Affiliation,42.009623,-87.670167,POINT (-87.67016685690001 42.0096228806),41.901207,-87.676356,POINT (-87.6763559892 41.90120699410001)
2,ee11a46ace3b156ce7f3e07d7d320cbca7160d1f,9ff153898c75dbd960a207b660a71f92f79b666dcec98e...,2013-08-19 16:15:00+00:00,2013-08-19 17:00:00+00:00,2160,0.0,1.703184e+10,1.703198e+10,32,76,...,2.0,44.85,Credit Card,Taxi Affiliation Services,41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),41.979071,-87.903040,POINT (-87.9030396611 41.9790708201)
3,6b904179e1c0913bea6d2eb3d5ecacab1df6aab4,4454ea9c7c01352d9c8826fb4cf9806dbf74137f776e94...,2013-08-19 15:45:00+00:00,2013-08-19 16:00:00+00:00,660,0.2,NaN,NaN,28,24,...,0.0,12.65,Credit Card,Taxi Affiliation Services,41.874005,-87.663518,POINT (-87.6635175498 41.874005383),41.901207,-87.676356,POINT (-87.6763559892 41.90120699410001)
4,a4cd80d477f8fd38ac80bd0687b87f4f170fef35,af5b394cb22984f667fc0a9cc30e0277cb9edf50f14747...,2013-08-11 11:15:00+00:00,2013-08-11 11:30:00+00:00,660,1.9,1.703108e+10,1.703184e+10,8,32,...,0.0,8.25,Cash,Dispatch Taxi Affiliation,41.899156,-87.626211,POINT (-87.6262105324 41.8991556134),41.880994,-87.632746,POINT (-87.6327464887 41.8809944707)


# Exercises

### 1) How can you predict the demand for taxis?

Say you work for a taxi company, and you're interested in predicting the demand for taxis.  Towards this goal, you'd like to create a plot that shows a rolling average of the daily number of taxi trips.  Amend the (partial) query below to return a DataFrame with two columns:
- `trip_date` - contains one entry for each date from January 1, 2016, to March 31, 2016.
- `avg_num_trips` - shows the average number of daily trips, calculated over a window including the value for the current date, along with the values for the preceding 3 days and the following 3 days, as long as the days fit within the three-month time frame.  For instance, when calculating the value in this column for January 3, 2016, the window will include the number of trips for the preceding 2 days, the current date, and the following 3 days.

This query is partially completed for you, and you need only write the part that calculates the `avg_num_trips` column.  Note that this query uses a common table expression (CTE); if you need to review how to use CTEs, you're encouraged to check out [this tutorial](https://www.kaggle.com/dansbecker/as-with) in the [Intro to SQL](https://www.kaggle.com/learn/intro-to-sql) course.

In [14]:
# Fill in the blank below
avg_num_trips_query = """
                      WITH trips_by_day AS
                      (
                      SELECT DATE(trip_start_timestamp) AS trip_date,
                          COUNT(*) as num_trips
                      
                      FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                      
                      WHERE trip_start_timestamp > '2016-01-01' AND trip_start_timestamp < '2016-04-01'
                      GROUP BY trip_date
                      ORDER BY trip_date
                      )
                      SELECT trip_date,
                          AVG(num_trips) 
                          OVER (
                               ORDER BY trip_date
                               ROWS BETWEEN 3 PRECEDING AND 3 FOLLOWING
                               ) AS avg_num_trips
                      FROM trips_by_day
                      """

# Check your answer
q_1.check()

,trip_date,avg_num_trips
0,2016-03-26,85243.142857
1,2016-03-28,78639.428571
2,2016-01-01,73067.250000
3,2016-01-15,84679.142857
4,2016-01-17,84366.285714


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [ ]:
# Lines below will give you a hint or solution code
# q_1.hint()
# q_1.solution()

### 2) Can you separate and order trips by community area?

The query below returns a DataFrame with three columns from the table: `pickup_community_area`, `trip_start_timestamp`, and `trip_end_timestamp`.  

Amend the query to return an additional column called `trip_number` which shows the order in which the trips were taken from their respective community areas.  So, the first trip of the day originating from community area 1 should receive a value of 1; the second trip of the day from the same area should receive a value of 2.  Likewise, the first trip of the day from community area 2 should receive a value of 1, and so on.

Note that there are many numbering functions that can be used to solve this problem (depending on how you want to deal with trips that started at the same time from the same community area); to answer this question, please use the **RANK()** function.

In [18]:
# Amend the query below
trip_number_query = """
                   SELECT pickup_community_area,
                        trip_start_timestamp,
                        trip_end_timestamp,
                        RANK()
                            OVER (
                                  PARTITION BY pickup_community_area
                                  ORDER BY trip_start_timestamp
                                 ) AS trip_number
                    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                    WHERE DATE(trip_start_timestamp) = '2013-10-03' 
                    """

# Check your answer
q_2.check()

,pickup_community_area,trip_start_timestamp,trip_end_timestamp,trip_number
0,31.0,2013-10-03 00:15:00+00:00,2013-10-03 00:15:00+00:00,1
1,31.0,2013-10-03 00:15:00+00:00,2013-10-03 00:15:00+00:00,1
2,31.0,2013-10-03 00:30:00+00:00,2013-10-03 00:45:00+00:00,3
3,31.0,2013-10-03 00:30:00+00:00,2013-10-03 00:30:00+00:00,3
4,31.0,2013-10-03 01:00:00+00:00,2013-10-03 01:30:00+00:00,5


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [19]:
# Lines below will give you a hint or solution code
#q_2.hint()
#q_2.solution()

### 3) How much time elapses between trips?

The (partial) query in the code cell below shows, for each trip in the selected time frame, the corresponding `taxi_id`, `trip_start_timestamp`, and `trip_end_timestamp`. 

Your task in this exercise is to edit the query to include an additional `prev_break` column that shows the length of the break (in minutes) that the driver had before each trip started (this corresponds to the time between `trip_start_timestamp` of the current trip and `trip_end_timestamp` of the previous trip).  Partition the calculation by `taxi_id`, and order the results within each partition by `trip_start_timestamp`.

Some sample results are shown below, where all rows correspond to the same driver (or `taxi_id`).  Take the time now to make sure that the values in the `prev_break` column make sense to you!

![first_commands](https://storage.googleapis.com/kaggle-media/learn/images/qjvQzg8.png)

Note that the first trip of the day for each driver should have a value of **NaN** (not a number) in the `prev_break` column.

In [16]:
# Fill in the blanks below
break_time_query = """
                   SELECT taxi_id,
                       trip_start_timestamp,
                       trip_end_timestamp,
                       TIMESTAMP_DIFF(
                           trip_start_timestamp, 
                           LAG(trip_end_timestamp, 1) OVER (PARTITION BY taxi_id ORDER BY trip_start_timestamp), 
                           MINUTE) as prev_break
                   FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                   WHERE DATE(trip_start_timestamp) = '2013-10-03' 
                   """

# Check your answer
q_3.check()

,taxi_id,trip_start_timestamp,trip_end_timestamp,prev_break
0,02f11605ad05d68d8135072d00215960282c16eb333e61...,2013-10-03 17:15:00+00:00,2013-10-03 17:15:00+00:00,225.0
1,0e21caaf401f961b7ae5a4c3d8c5b70291ccc2b6c2138c...,2013-10-03 10:00:00+00:00,2013-10-03 10:00:00+00:00,-45.0
2,137a3605d33badaf4f3ced9115d44ebd822c90ec3cedcb...,2013-10-03 16:45:00+00:00,2013-10-03 16:30:00+00:00,165.0
3,1f449840a00f8ff9ae6f872257982970a71b322f93fee8...,2013-10-03 13:30:00+00:00,2013-10-03 13:30:00+00:00,165.0
4,1f449840a00f8ff9ae6f872257982970a71b322f93fee8...,2013-10-03 22:15:00+00:00,2013-10-03 22:15:00+00:00,165.0


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [ ]:
# Lines below will give you a hint or solution code
#q_3.hint()
#q_3.solution()

# Keep going

Move on to learn how to query **[nested and repeated data](https://www.kaggle.com/alexisbcook/nested-and-repeated-data)**.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/advanced-sql/discussion) to chat with other learners.*